In [1]:
import cv2
import numpy as np
import time

In [3]:
#warming up the camera
cap=cv2.VideoCapture(0)
time.sleep(2)

In [4]:
#recording the background
for i in range(30):
    ret,bg=cap.read()
print('Initial BG done')
bg=cv2.flip(bg,1)

Initial BG done


In [5]:
#for colors needing 2 masks like red
'''lower_red1=np.array([0,120,70])
upper_red1=np.array([10,255,255])
lower_red2=np.array([170,120,70])
upper_red2=np.array([180,255,255])'''

#values can be changed for different colors
#current values are for yellow
lower_color=np.array([15,100,100])
upper_color=np.array([35,255,255])

In [6]:
#for fine tuning HSV we can use trackbars
'''def nothing(x):
    pass
cv2.namedWindow('trackbars')
cv2.createTrackbar('lower H','trackbars',0,179,nothing)
cv2.createTrackbar('lower S','trackbars',0,255,nothing)
cv2.createTrackbar('lower V','trackbars',0,255,nothing)

cv2.createTrackbar('upper H','trackbars',0,179,nothing)
cv2.createTrackbar('upper S','trackbars',255,255,nothing)
cv2.createTrackbar('upper V','trackbars',255,255,nothing)'''

"def nothing(x):\n    pass\ncv2.namedWindow('trackbars')\ncv2.createTrackbar('lower H','trackbars',0,179,nothing)\ncv2.createTrackbar('lower S','trackbars',0,255,nothing)\ncv2.createTrackbar('lower V','trackbars',0,255,nothing)\n\ncv2.createTrackbar('upper H','trackbars',0,179,nothing)\ncv2.createTrackbar('upper S','trackbars',255,255,nothing)\ncv2.createTrackbar('upper V','trackbars',255,255,nothing)"

In [7]:
while cap.isOpened():
    ret,frame=cap.read()
    if not ret:
        break
    frame=cv2.flip(frame,1)
    #changing BGR to HSV
    hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)

    #for colors needing 2 masks like red
    '''mask1=cv2.inRange(hsv,lower_red1,upper_red1)
    mask2=cv2.inRange(hsv,lower_red2,upper_red2)
    mask=mask1+mask2'''

    #for trackbars
    '''lower_h=cv2.getTrackbarPos('lower H','trackbars')
    lower_s=cv2.getTrackbarPos('lower S','trackbars')
    lower_v=cv2.getTrackbarPos('lower V','trackbars')

    upper_h=cv2.getTrackbarPos('upper H','trackbars')
    upper_s=cv2.getTrackbarPos('upper S','trackbars')
    upper_v=cv2.getTrackbarPos('upper V','trackbars')

    lower_bound=np.array([lower_h,lower_s,lower_v])
    upper_bound=np.array([upper_h,upper_s,upper_v])

    mask=cv2.inRange(hsv,lower_bound,upper_bound)'''

    #creating mask using the lower bound and upper bound of the color
    mask=cv2.inRange(hsv,lower_color,upper_color)

    #morphing the masks to reduce noise
    mask=cv2.morphologyEx(mask,cv2.MORPH_OPEN,np.ones((5,5),np.uint8))
    mask=cv2.morphologyEx(mask,cv2.MORPH_DILATE,np.ones((5,5),np.uint8))

    #inverse mask to get remaining part of the frame
    mask_inv=cv2.bitwise_not(mask)

    #background where cloak is detected
    cloak=cv2.bitwise_and(bg,bg,mask=mask)

    #remaining part of the prame
    remaining=cv2.bitwise_and(frame,frame,mask=mask_inv)

    #combine both areas
    output=cv2.addWeighted(cloak,1,remaining,1,0)

    #display results
    cv2.imshow('Invisibiliy Cloak',output)
    cv2.imshow('original',frame)

    if cv2.waitKey(1)==27:
        break